# Ross Ice Shelf, Antarctica

**Station Name:**  lorg

**Location:**  Lorne, Antarctica

**Archive:**  [UNAVCO](http://www.unavco.org)

**DOI:**  [https://doi.org/10.7283/ACF6-YT23](https://doi.org/10.7283/ACF6-YT23)

**Ellipsoidal Coordinates:**

- Latitude:  -78.18365
- Longitude: 170.03361
- Height:  	-7.778 m

[Station Page at UNAVCO](https://www.unavco.org/instrumentation/networks/status/nota/overview/lorg)

[Station Page at Nevada Geodetic Laboratory](http://geodesy.unr.edu/NGLStationPages/stations/LORG.sta)

[Google Maps Link](https://goo.gl/maps/bSAuLXLLMmzWqPdW9) 

<img src="https://gnss-reflections.org/static/images/LORG.jpg" width="500">



### Data Summary

Station lorg is on the Ross Ice Shelf in Antarctica.
The site is a largely featureless ice plain with little to no obstructions. 
It was installed November 2018 and decommissioned and removed in 2019 and it recorded only GPS frequencies during its operation. 

LORG is an example station on the GNSSIR webapp.

### Imports

In [ ]:
import json
import os
import sys

import ipywidgets as widgets
import seaborn as sns

sns.set_theme(style="whitegrid");

# We are including our repository bin to the system path so that we can import the following python modules
bin_path = os.path.abspath(os.path.join('../../../bin'))
if bin_path not in sys.path:
    sys.path.append(bin_path)

import gnssrefl_helpers

#Making sure environment variables are set - this is required to run the gnssrefl code
exists = gnssrefl_helpers.check_environment()
if exists == False:
    gnssrefl_helpers.set_environment(refl_code="../../..", orbits="../../../orbits", exe="../../../bin/exe")

# Set local variable of refl_code location
refl_code_loc = os.environ['REFL_CODE']

# import gnssrefl functions
from gnssrefl.rinex2snr_cl import rinex2snr
from gnssrefl.quickLook_cl import quicklook
from gnssrefl.make_json_input import make_json
from gnssrefl.gnssir_cl import gnssir
from gnssrefl.daily_avg_cl import daily_avg
from gnssrefl.installexe_cl import installexe

#@formatter:off
%matplotlib inline

In [ ]:
# import the crx2rnx file which is dependant on your working OS - this is required to run the gnssrefl code
# If in docker environment, then you do not need to download crxnrnx
try:
    os.environ['DOCKER']
except KeyError:
    sys = gnssrefl_helpers.get_sys()
    installexe(sys)

Fast Mode - this will download data weekly instead of daily

In [ ]:
weekly = widgets.Checkbox(value=True, description='Fast Mode', disabled=False, indent=False)
display(weekly)

## Take a look at the Data


First, we'll make a SNR file. Let's choose day of year 205 in the year 2019 to look at. We'll also save the lat and long information as variables to be used later during analysis.

In [ ]:
station = 'lorg'
year = 2019
doy = 205

lat = -78.1836
long = 170.0336
height = -7.722

In [ ]:
rinex2snr(station, year, doy)

Use **quickLook** to produce a periodogram - quickLook is set to use the L1 frequency by default:

In [ ]:
values, metrics = quicklook(station, year, doy=doy)

This looks pretty good. It looks like we will not need an azimuth mask - which makes sense as there are no significant topographic features near the station and so we will use default values for the elevation mask and an azimuth mask is not required.

We will now compare the periodograms for other frequencies: L2C and L5.

This is for L2C:

In [ ]:
values, metrics = quicklook(station, year, doy=doy, fr=20)

This is for L5:

In [ ]:
values, metrics = quicklook(station, year, doy=doy, fr=5)

They do look pretty similar to the L1 periodogram, except that there
are fewer satellite traces because only GPS satellites launched after 2005
broadcast L2C and only satellites after 2010 broadcast L5.

### Analyze the Data

So now we prepare to analyze the data using **gnssir**.
First ,we create our set of analysis instructions with make json.
The default settings only need the station name, latitude, longitude, and ellipsoidal height.

In [ ]:
# we saved the lat, long, and height earlier in the notebook
make_json(station, lat, long, height)

In [ ]:
# This is the json file that was created
json_file = f'{refl_code_loc}/input/lorg.json'
with open(json_file, "r") as myfile:
    file = json.load(myfile)

file

Next make some snr files for a time span of about eight months. We will do day of year 1 through day of year 233 for the year 2019. We can also restrict the search to the UNAVCO archive to make the
code run faster (otherwise it will check other archives as well).

In [ ]:
rinex2snr(station, year=2019, doy=1, doy_end=233, archive='unavco', weekly=weekly.value)

We would then run **gnssir** for the same time range. 

In [ ]:
doy = 1
doy_end = 233
gnssir(station, year, doy=doy, doy_end=doy_end, screenstats=False)

Then the **daily_avg** command will calculate the daily average reflector height from the daily output files. To minimize outliers in these daily averages, we'll set a median filter to 0.25 meters and the required minimum number of daily satellite tracks we will set to 50.

In [ ]:
daily_avg(station, medfilter=.25, ReqTracks=50, plt=False, txtfile='lorg-dailyavg.txt')

This shows the range of reflector heights each day, with each day represented by a different color.

This is not quite perfect - there are some outliers. But it is an great example of how easy it is to use the default code to estimate snow accumulation in Antarctica.

These reflector heights then give information about snow accumulation at lorg. 

We have reversed the y-axis so that the reflector height gets smaller as the snow layer increases and the distance between the antenna and snow surface decreases.